### Requirements
pandas              1.4.3  <br>
requests            2.28.1 <br>
psycopg2            2.9.3 <br>


# OS Downloads API - Open Data


In [2]:
import os
import pandas as pd
import requests as r
import json
import hashlib
import zipfile
import psycopg2

In [3]:
#Get the result of the OS Downloads API call 
open_prod = r.get('https://api.os.uk/downloads/v1/products').json()

In [4]:
#Load view into DataFrame 
df = pd.DataFrame(open_prod)
df

,id,name,description,version,url
0,250kScaleColourRaster,1:250 000 Scale Colour Raster™,"Get the regional view of towns and villages, r...",2022-06,https://api.os.uk/downloads/v1/products/250kSc...
1,BoundaryLine,Boundary-Line™,"From Euro constituencies to council wards, Bou...",2022-10,https://api.os.uk/downloads/v1/products/Bounda...
2,CodePointOpen,Code-Point® Open,Free and open postcode location data. Can be u...,2022-11,https://api.os.uk/downloads/v1/products/CodePo...
3,GBOverviewMaps,GB Overview Maps,Our simplest maps of the British Isles.,2014-11,https://api.os.uk/downloads/v1/products/GBOver...
4,LIDS,OS Open Linked Identifiers,A comprehensive dataset of cross-referenced id...,2022-09,https://api.os.uk/downloads/v1/products/LIDS
5,MiniScale,MiniScale®,A simple overview map of Great Britain.,2022-01,https://api.os.uk/downloads/v1/products/MiniScale
6,OpenGreenspace,OS Open Greenspace,Covering a range of greenspaces in urban and r...,2022-10,https://api.os.uk/downloads/v1/products/OpenGr...
7,OpenMapLocal,OS OpenMap - Local,"Map, visualise and truly understand your data ...",2022-04,https://api.os.uk/downloads/v1/products/OpenMa...
8,OpenNames,OS Open Names,"A comprehensive dataset of place names, roads ...",2022-10,https://api.os.uk/downloads/v1/products/OpenNames
9,OpenRivers,OS Open Rivers,Understand how watercourses in Great Britain j...,2022-10,https://api.os.uk/downloads/v1/products/OpenRi...


In [13]:
#define download location
download_loc = r''

In [14]:
#create download function
def download_file(url,file):
    with r.get(url, stream=True) as d:
        d.raise_for_status()
        with open(download_loc+'/'+file, 'wb') as f:
            for chunk in d.iter_content(chunk_size=8192): 
                f.write(chunk)
    return download_loc

## What is MD5?
MD5 (message-digest algorithm) is a cryptographic protocol used for authenticating messages as well as content verification and digital signatures. MD5 is based on a hash function that verifies that a file you sent matches the file received by the person you sent it to. 

In [15]:
#create md5 function
def md5(fname):
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [16]:
#define our product of interest
open_product = 'OpenRivers'

In [17]:
df = df.loc[df['id'] == open_product] #locate the correct product
product = r.get(df.iloc[0]['url']).json() #get URL request for product
download = product['downloadsUrl'] #find the download URL 
filetypes = r.get(download).json() #get filetypes with download URLs
pd.DataFrame(filetypes) 

,md5,size,url,format,area,fileName,subformat
0,765d0528d8c685567790877042b9ee85,39114249,https://api.os.uk/downloads/v1/products/OpenRi...,ESRI® Shapefile,GB,oprvrs_essh_gb.zip,NaN
1,270abd3b058a193eb092cf08023e7abb,40248959,https://api.os.uk/downloads/v1/products/OpenRi...,GML,GB,oprvrs_gml3_gb.zip,3
2,27a054fc093c14f6bfe46494b5f74e15,55042488,https://api.os.uk/downloads/v1/products/OpenRi...,GeoPackage,GB,oprvrs_gpkg_gb.zip,NaN
3,d937869f756c1c1bf2f1670dedce6b36,131121463,https://api.os.uk/downloads/v1/products/OpenRi...,Vector Tiles,GB,oprvrs_mbtiles_gb.zip,(MBTiles)


In [18]:
#define desired filetype
download_filetype = 'GeoPackage'

In [19]:
#find the right filetype
for x in range(len(filetypes)):
    if filetypes[x]['format'] == download_filetype:
        #download the file
        download_file(filetypes[x]['url'],filetypes[x]['fileName'])
        #check the file md5's match
        if md5(download_loc +'/'+ filetypes[x]['fileName']) == filetypes[x]['md5']:
            print('Successfully Downloaded ')
            file_path = download_loc +'\\'+ filetypes[x]['fileName']
        else: 
            print('MD5 does not match')


Successfully Downloaded 


In [20]:
#unzip the file
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(download_loc+'\\'+open_product)
path = os.listdir(download_loc+'\\'+open_product +'\\' + 'data')
file = download_loc+'\\'+open_product +'\\' + 'data' + '\\' + path[0]


In [22]:
#create schema
#if using Linux use getpass.getuser() as the variable in schema creation

#connection details
dbname = 'foss4g'
host = '18.135.248.161'
port = '5432'
user = 'postgres'
password = 'V2ql011Bw5A6'

conn_local = psycopg2.connect(database="postgres", user=f"postgres", password=f"password",host="localhost")
#conn_cloud = psycopg2.connect(database=dbname, user=user, password=password,host=host)
cursor = conn_local.cursor()
sql = f'''CREATE SCHEMA IF NOT EXISTS {os.getlogin( )}; '''
cursor.execute(sql)
conn_local.commit()
conn_local.close()

In [13]:
#create schema if you have postgresql installed  
#if using Linux use getpass.getuser() as the variable in schema creation
pg_connection = '-d postgres -U postgres -p password -h localhost -p 5432'
create_schema = f'''psql {pg_connection} -c "CREATE SCHEMA IF NOT EXISTS {os.getlogin( )}"'''
print(create_schema)
os.system(create_schema)

psql -d postgres -U postgres -p password -h localhost -p 5432 -c "CREATE SCHEMA IF NOT EXISTS dchapman"


0

In [20]:
#load data if you have ogr2ogr
#if using Linux use getpass.getuser() as the variable in active_schema 
pg_connection_ogr = f'PG:"dbname=postgres user=postgres password=password active_schema={os.getlogin( )} host=localhost port=5432"'
#pg_connection_ogr = f'PG:"dbname={dbname} user={user} password={password} active_schema={os.getlogin( )} host={host} port={port}"'
command = f'ogr2ogr -f PostgreSQL {pg_connection_ogr} {file}'
print(command)
os.system(command)


ogr2ogr -f PostgreSQL PG:"dbname=postgres user=postgres password=password active_schema=dchapman host=localhost port=5432" C:\Users\dchapman\temp\OpenRivers\data\oprvrs_gb.gpkg


0